In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.transforms import transforms

torch.set_printoptions(linewidth=120)

from torch.utils.tensorboard import SummaryWriter


In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        t = self.out(t)

        return t


In [4]:
train_set = torchvision.datasets.FashionMNIST(root='/.data/', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))


In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

###      starting out with tensorboard (Network graph and images)

In [6]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))   
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

# tensorboard - -logdir = ./runs/ ---> to run tensorboard


# Training loop review

In [7]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)


for epoch in range(1):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f'epoch {epoch}, total loss: {total_loss}, total correct: {total_correct}')


epoch 0, total loss: 365.98906630277634, total correct: 45962


In [26]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:  # Get Batch

        images, labels = batch
        preds = network(images)
        # Calculate Loss
        loss = F.cross_entropy(preds, labels)
        # Calculate Gradient
        optimizer.zero_grad()
        loss.backward()
        
        # Update Weights
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)

    print("epoch", epoch,"total_correct:", total_correct,"loss:", total_loss)

tb.close()


epoch 0 total_correct: 47529 loss: 332.54088458418846
epoch 1 total_correct: 51556 loss: 227.78512583673
epoch 2 total_correct: 52207 loss: 211.19314129650593
epoch 3 total_correct: 52589 loss: 202.38977833092213
epoch 4 total_correct: 52770 loss: 196.7905191630125
epoch 5 total_correct: 53001 loss: 190.3728225827217
epoch 6 total_correct: 53128 loss: 188.9839306473732
epoch 7 total_correct: 53184 loss: 186.22973096370697
epoch 8 total_correct: 53164 loss: 186.81141820549965
epoch 9 total_correct: 53282 loss: 184.24816250056028
